In [2]:
import socket
import time

# open socket connection as a client
with socket.socket(socket.AF_INET, socket.SOCK_DGRAM, ) as s:
    # connect to the server
    s.settimeout(5)
    s.bind(("0.0.0.0", 56790))

    while True:
        data, addr = s.recvfrom(8192)
        if '>' in data.decode():
            print(f"{time.strftime("%Y-%m-%d %H:%M:%S")} {data.decode()}")
        else:
            print(f"{time.strftime("%Y-%m-%d %H:%M:%S")} no '>' in data or no data received")

TimeoutError: timed out

In [ ]:
from nrbdaq.instr.fidas import FIDAS
from nrbdaq.utils.utils import load_config

config = load_config(config_file="nrbdaq.yml")
with FIDAS(config=config) as fidas:
    if fidas:
        fidas.run()


In [ ]:
import os
import polars as pl

source = "tests/data/fidas"

fidas_data_fields = {
    0: "status bit sensor flow",
    1: "status bit coincidence",
    2: "status bit suction pumps",
    3: "status bit weather station",
    4: "status bit IADS",
    5: "status bit estimated raw channel deviation",
    6: "status bit LED temperature",
    7: "status bit operating modus",
    20: "velocity [m/s]",
    21: "coincidence [%]",
    22: "modus",
    23: "suction pump output [%]",
    24: "IADS temperature (Fidas) evaporation unit (UF-CPC), sensor #1 (Promo) [°C]",
    25: "estimated raw channel deviation [channels]",
    26: "LED temperature [°C]",
    27: "flow rate [l/min]",
    28: "Cn for UF-CPC [P/cm³] (count and nephelometer modus)",
    29: "x50 droplet diameter (UF-CPC) [μm]",
    30: "temperature of condensation unit (UF-CPC), sensor #2 (Promo) [°C]",
    40: "temperature [°C]",
    41: "relative humidity [%]",
    42: "wind speed [km/h]",
    43: "wind direction [°]",
    44: "precipitation intensity [l/m²/h]",
    45: "precipitation type",
    46: "temperature dew point [°C]",
    47: "air pressure [hPa]",
    48: "wind signal quality [%]",
    # Fidas/Promo only:
    52: "PM2.5 [mg/m³] - 1 s average",
    53: "PM10 [mg/m³] - 1 s average",
    54: "PM1 [mg/m³] - 10 s average",
    55: "PM2.5 [mg/m³] - 10 s average",
    56: "PM10 [mg/m³] - 10 s average",
    57: "PMtot [mg/m³] - 10 s average",
    58: "PM2.5 [mg/m³] - 60 s average",
    59: "PM10 [mg/m³] - 60 s average",
    60: "Cn [P/cm³] (PM averaging interval, default: 900s)",
    61: "PM1 [mg/m³]",
    62: "PM2.5 [mg/m³]",
    63: "PM4 [mg/m³]",
    64: "PM10 [mg/m³]",
    65: "PMtotal [mg/m³]",
    "66-109": "further PM values [mg/m³] (different algorithms)",
    "110ff": "ΔCn [P/cm³] size distribution with size intervals as shown by the device under Expert User Mode / Particle Size Distribution / Table (10 s average)",
}
df = pl.DataFrame()
for root, dirs, files in os.walk(source):
    for file in files:
        src=os.path.join(root, file)
        _ = pl.read_parquet(src)
        # _ = _.with_columns(pl.col('dtm').str.to_datetime(format='%y-%m-%ddd %H:%M:%S'))
        print(f"{src} is of shape: {_.shape}")
        df = pl.concat([df, _], how='diagonal').sort('dtm')

display(df.describe())

field = '59'
chart = (df.plot.line(x='dtm', y=field)).properties(width=500, title="FIDAS NRB")
chart.encoding.x.title = "dtm"
chart.encoding.y.title = fidas_data_fields[field]
chart
